# 2018-01-22 - Mendeley, mémoire
---
J'ai emprunté à la BU Timone trois livres qui me serviront de sources notamment pour mon introduction, ma discussion et ma conclusion : 
+ "Neuromathematics of Vision" de G. Citti et A. S. Sarti (2014)
+ "Understanding Vision - Theory, models and data" de Li Zhaoping (2014)
+ "The new visual neurosciences" édité par J. S. Werner et L. M. Chapula (2014)  

Ce sont les livres les plus complets et les plus récents que j'ai pu trouver en emprun physique.  
Le dernier est particulièrement imposant (~1700 pages) et est édité par le MIT. 

# 2018-01-23 - TensorBoard
---
Je me suis battu tout la journée pour faire fonctionner TensorBoard (en suivant ce [tuto](https://www.tensorflow.org/get_started/summaries_and_tensorboard)). Pour une raison que j'ignore, lors de son invocation (dans une console) via

    tensorboard --logdir "/tmp/tensorflow/mnist/logs/mnist_with_summaries"
    
Apparait un conflit avec numpy.multiarray :

    (...)
    Importing the multiarray numpy extension module failed
    (...)
    Original error was: /usr/local/lib/python2.7/dist-packages/numpy/core/multiarray.so: wrong ELF class: ELFCLASS64

Le problème semble provenir de la version de Python liée à TensorBoard (2.7 au lieu de 3.+).
Les seules indices que j'ai trouvé pour résoudre ce problème sont dans ces deux liens : [1](https://stackoverflow.com/questions/39342920/my-numpy-is-latest-but-tensorboard-says-its-old-one), [2](https://stackoverflow.com/questions/46118952/cant-launch-tensorboard-from-cmd).

Lorsqu'il utilise TensorFlow, Daucé n'utilise que rarement TensorBoard et extrait directement les variables pour créer ses visuels. Pour me caler sur cette méthode, je dois traduire mon script haut-niveau en script bas-niveau.  

La fonction 

    estimator.get_variable_names()
    
ne semble fonctionner qu'après avoir réaliser l'entrainement, donc elle ne permettrait pas de récupérer les poids au fil de celui-ci pour réaliser des graphiques.  
Plusieurs variables semblent correspondre à des poids : 

    > 'global_step', 
    > 'linear/linear_model/bias_weights'
    > 'linear/linear_model/bias_weights/part_0/Ftrl' 
    > 'linear/linear_model/bias_weights/part_0/Ftrl_1' 
    > 'linear/linear_model/x/weights'
    > 'linear/linear_model/x/weights/part_0/Ftrl' 
    > 'linear/linear_model/x/weights/part_0/Ftrl_1'
    
Mais ces variables semblent s'imprimer correctement avec 

    estimator.get_variable_value('linear/linear_model/x/weights')

# 2018-01-24 - LaTeX débug, TensorFlow bas-niveau
---
Alors que ma compilation fonctionnait bien jusqu'à maintenant, depuis avant-hier, celle-ci me redemande des packages que j'ai déjà installé (etoolbox). Lorsque je retente de les installer, j'obtiens l'erreur : 

    >The following packages have unmet dependencies:
    > mendeleydesktop : Depends: python but it is not going to be installed
    > texlive-pictures : Depends: python but it is not going to be installed

Il semble y avoir une erreur avec mon installation python?  
En lancant 

    apt-get -f install
    
    (...)
    >The following additional packages will be installed:
    >  python python-minimal python2.7 python2.7-minimal
    >Suggested packages:
    >  python-doc python-tk python2.7-doc binfmt-support
    >The following packages will be REMOVED:
    >  python:i386 python-minimal:i386 python2.7:i386 python2.7-minimal:i386
    >The following NEW packages will be installed:
    >  python python-minimal python2.7 python2.7-minimal
    (...)
    >Setting up python2.7 (2.7.12-1ubuntu0~16.04.3) ...
    >Setting up python (2.7.11-1) ...
    >Setting up mendeleydesktop (1.17.13) ...
    >Setting up Mendeley Desktop apt repository
    (...)
    
Après ça, les lignes

    apt-get install etoolbox
    apt-get install texlive-science
    
fonctionnent, ainsi que la compilation de mon fichier.   
Mais pourquoi est-ce que j'ai soudain dû re-installer mes packages LaTeX? La seule chose que j'ai fait entre temps était d'installer Mendeley. Comment Mendeley peut-il influer sur ces packages?  

Nouveau problème : ma bibliographie ne s'affiche pas.  
La solution était simple. Seules les sources citées sont affichées dans la bibliographie. Je dois aussi lancer deux fois de suite pdfLaTeX.  

---

J'ai un problème dans l'écriture de la régression linéaire bas-niveau dans le script Waveimage_TensorFlow.  
J'ai créé un placeholder pour mes données :

    x_train = tf.placeholder(tf.float32, shape=[batch_size, 76])
    
Donc je veux remplir les lignes les unes après les autres à l'aide de :

    batch = mnist.test.next_batch(batch_size)
    for x in batch[0]:
        i_offset, j_offset = minmax(int(np.random.randn() * 15), 40), minmax(int(np.random.randn() * 15), 40)  
        v, w = generate_vector_data_with_offset_from_x(x, i_offset, j_offset)
        
        sess.run(x_train, feed_dict={x_train[:,num_row]: v})

Avec num_row correspondant au numéro de la ligne à remplir avec le vecteur v (comprenant les informations de l'image transformée). Mais j'enchaine erreur sur erreur.  

La seule chose qui semble me rapprocher d'une solution, est de d'abort créer des coupes de x_train, pour pouvoir les remplir individuellement dans ma boucle for : 

    xslice = tf.placeholder(tf.int32)
    x_train_slice = tf.slice(x_train, xslice, size=[64])
    
Mais ces lignes sortent un certain nombre d'erreurs, dont :

    InvalidArgumentError: Shape must be rank 1 but is rank 2 for 'Slice_9' (op: 'Slice') with input shapes: [3,76], ?, [64].
    
Pour l'instant je ne comprends pas bien comment résoudre tout ça (je me base sur ce [lien](https://stackoverflow.com/questions/34002591/tensorflow-slicing-based-on-variable) et l'[entrée API](https://www.tensorflow.org/versions/master/api_docs/python/tf/slice) de la fonction).

# 2018-01-25 - TensorFlow bas-niveau, tranches de données
---
J'ai un peu avancé sur mon problème d'assignation de données

    xslice = tf.placeholder(tf.int32)
    sess.run(tf.global_variables_initializer())
    
    n = [2,10]
    sess.run(xslice, feed_dict={xslice: n})
    
    >array([ 2, 10], dtype=int32)
    
Donc j'arrive à assigner des valeurs de cette façon. Mais lorsque je tente de réaliser des itérations pour modifier les valeurs en vol :

    for n in range(10):
        sess.run(xslice, feed_dict={xslice: n})
        
Je n'ai aucune sortie, ~~aucune valeur ne semble être attribuée au placeholder.~~  
Il suffisait d'utiliser print(): 

    for n in range(2):
    sess.run(xslice, feed_dict={xslice: [0,n]})
    > [0 0]
    > [0 1]
    > [0 2]
    
Je bloque toujours sur l'assignement en direct sur des tranches de placeholder : 

    x_train = tf.placeholder(tf.float32, shape=[batch_size, 76])
    slice_begin = tf.placeholder(tf.int32)
    x_train_slice = tf.slice(x_train, begin=slice_begin, size=[1,76])
    
    sess.run(tf.global_variables_initializer())

    data = np.random.randn(batch_size, 76)
    
    for n in range(3):
        sess.run(x_train_slice, feed_dict={x_train: data, slice_begin: [n,0]})
        
Ce bloc assigne l'ensemble de *data* à *x_train* et récupère les tranches les unes après les autres (correspondant à chaque entrée, cad un vecteur de 76 valeurs) pour les lire/afficher.  
Ca ne corresponds pas à ce que je veux faire. Je veux à chaque itération générer des données, et attribuer ces données à une couche spécifique de *x_train* : 
    
    for n in range(3):
        data = np.random.randn(1, 76)
        sess.run(x_train_slice, feed_dict={x_train[n]: data, slice_begin: [n,0]})
        
Je ne peux pas utiliser la fonction [tf.assign](https://www.tensorflow.org/api_docs/python/tf/assign) qui ne fonctionne qu'avec des tenseurs Variables.  
J'ai tout de même trouvé quelques indices [ici](https://stackoverflow.com/questions/39157723/how-to-do-slice-assignment-in-tensorflow/43139565#43139565) et [là](https://stackoverflow.com/questions/42778913/assign-values-to-an-array-in-a-loop-in-tensorflow) qui pourraient mener vers une solution. Il faut aussi que je me penche sur cette partie de l'API : [Importing data](https://www.tensorflow.org/programmers_guide/datasets#Feeding).

# 2018-01-26
---
Quelques ressources LaTeX :
- [Not So Short Introduction to LaTeX](http://www.ctan.org/tex-archive/info/lshort/english/lshort.pdf)
- [A Short Math Guide to LaTeX](ftp://ftp.ams.org/pub/tex/doc/amsmath/short-math-guide.pdf)
- [The Comprehensive LaTeX~Symbol List](http://ctan.org/pkg/comprehensive)
- [Most commonly used math symbols](http://www.sunilpatel.co.uk/latex-type/latex-math-symbols/)

---
# To Do
+ ~~Se renseigner sur Mendeley pour gérer la biblio~~
+ ~~Installer et initialiser Mendeley~~
+ Partager le groupe Mendely avec Daucé et Perrinet
+ S'inscrire sur la liste de diffusion de l'INS
### Waveimage_TensorFlow
+ Traduire le script de régression linéaire haut-niveau en script bas-niveau
    + Résoudre le problème d'assignation des données à x_train
+ ~~Tester la ligne de code "estimator.get_variable_names()"~~
### Waveimage-tensorflow-pow3-encoding_P3ver
+ Créer un graphique : nombre de saccades nécessaires à atteindre la cible en fonction de sa distance du point de fixation
+ ~~Réaliser à nouveau les benchmarkings du paramètre alpha : le titre débordait de la figure~~
+ Réaliser une transformation coordonnée -> degrés et adapter le modèle pour les utiliser
### mnist-logPolar-encoding
+ Ecrire un script réalisant un apprentissage par régression linéaire en utilisant les sorties de la partie "Energy"
+ Intégrer un bruit écologique aux images (cf Najemnik2005 (chercher la méthode utilisée dans les sources); librairie [SLIP](https://nbviewer.jupyter.org/github/bicv/SLIP/blob/master/SLIP.ipynb) de Laurent)
### Rapport M2a
+ ~~Construire le squelette .tex du rapport~~
+ ~~Résoudre le problème de compilation~~
+ ~~Résoudre le problème d'affiche de la bibliographie~~
+ Ecrire une ébauche d'introduction

---
# A lire
+ http://bethgelab.org/media/publications/Kuemmerer_High_Low_Level_Fixations_ICCV_2017.pdf
+ https://pdfs.semanticscholar.org/0182/5573781674bcf85d0f5d2ec456842f75ad3c.pdf
+ Schmidhuber, 1991 (voir mail Daucé)
+ Parr and Friston, 2017 (voir mail Perrinet)
+ http://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1003005#s1  
### Magnocellular pathway function  
+ [Selective suppression of the magnocellular visual pathway during saccadic eye movements](http://www.nature.com.lama.univ-amu.fr/articles/371511a0), Burr1994
+ [On Identifying Magnocellular and Parvocellular Responses on the Basis of Contrast-Response Functions](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3004196/), Skottun2011
+ [Review: Steady and pulsed pedestals, the how and why of post-receptoral pathway separation](http://jov.arvojournals.org/article.aspx?articleid=2191890), Pokorny2011
+ [An evolving view of duplex vision: separate but interacting cortical pathways for perception and action](http://www.sciencedirect.com/science/article/pii/S0959438804000340?via%3Dihub), Goodale2004
+ [Quantitative measurement of saccade amplitude, duration, and velocity](http://n.neurology.org/content/25/11/1065), Baloh1975

---
# Satellites
- [Cooperating with machines](https://www.nature.com/articles/s41467-017-02597-8); Crandall et al., 2018
- [Predicting Public Corruption with Neural Networks: An Analysis of Spanish Provinces](https://link.springer.com/article/10.1007/s11205-017-1802-2) ; F. J. López-Iturriaga and I. P. Sanz, 2017
- [SiGe epitaxial memory for neuromorphic computing with reproducible high performance based on engineered dislocations](https://www.nature.com/articles/s41563-017-0001-5) ; S. Choi et al., 2018